In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
from ese.analysis.analyze_inf import load_cal_inference_stats
from ese.analysis.analysis_utils.plot_utils import get_prop_color_palette
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: '/storage/vbutoi/scratch/ESE/inference/10_09_24_OCTA_Benchmark_Exps'
    inference_groups: 
        # - 'Threshold_Sweep_CrossEntropy'
        # - 'Threshold_Sweep_Dice'
        - 'Temperature_Sweep_CrossEntropy'
        - 'Temperature_Sweep_Dice'

options:
    verify_graceful_exit: True
    equal_rows_per_cfg_assert: False 

In [ ]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=False
)

In [4]:
# for ikey in inference_df.keys():
#     print(ikey)

In [5]:
cols_to_keep = [
    'abs_soft_volume_error',
    'log_soft_volume_error',
    'abs_hard_volume_error',
    'log_hard_volume_error',
    'loss_func_class',
    'temperature',
    'gt_volume',
    'threshold',
    'data_id',
    'split'
]
# Filter out the columns we want to keep
exp_df = inference_df[cols_to_keep].drop_duplicates()

In [ ]:
plt.figure(figsize=(30, 20))
# We want to plot the mean error vs temperature
g = sns.relplot(
    data=exp_df,
    # x='threshold',
    # y='log_hard_volume_error',
    x='temperature',
    y='log_soft_volume_error',
    hue='loss_func_class',
    col='split',
    kind='line',
    height=10,
    aspect=2,
)

# g.set(xticks=np.arange(0, 1.1, 0.1))
g.set(xticks=np.arange(0, 3.0, 0.1))

In [ ]:
# We want to plot the mean error vs temperature
g = sns.relplot(
    data=exp_df,
    # x='threshold',
    # y='log_hard_volume_error',
    x='temperature',
    y='log_soft_volume_error',
    hue='gt_volume',
    col = 'loss_func_class',
    row ='split',
    kind='line',
    height=8,
    aspect=2.0,
    legend=False,
    facet_kws={'sharex': False},
    palette=get_prop_color_palette(exp_df, 'gt_volume')
)

g.set(xticks=np.arange(0, 3.0, 0.1))